In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'faster-rcnn-resnet50-fpn:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F809784%2F1387311%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T095727Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D830d71d21d3e20efe37b4d8de89df9f31e696e648bfe3708dd942240b5e9bbcde7dc9680f3adc9dd7ea90525d34b1986272678d5a871ff8d6ebddd614a87d15a37745d7613b84d3383f9bec5b116e09f564fca9694abc42c24a05c6440e75c639babfbb0175a03b4d34cae29b6a049a3716d0203d374134d26b942815849c71bfe00b46860311c31198fe33218d1b1130ee6d2fbf7c4f96704bd277c8a13f7d5fbc3f71f878d5fa4d81d291c212bb8c8993fa440fc09fdc505e79dadfff6ae6275af407c46457b51c0116771446244a7ac5203dea538a5646b2d35cc003f85b2cca53cc4dd27411b1281f2c6bf9d943e27cd37758cf2fd88ad77562ce5b1dc9b,anomaly-detection:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4489285%2F7692246%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240817%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240817T095727Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3aa975fb5a3e01078608bd17e40ab995391eafd8fd5cf5f46c7f3470784ac7d7fb4df0b4f760e4ffcb0ad3eea17a358fc4cb2d9ab27bfbceebcbc4f33541dc43aa1419e9e026232c0ffd18b3735918410b45716852d25ac098b43f4364d6c19ee81ecad56fa727030af0865c60dc79655753ce810464933c1db64a5131f798e9857a6639682b3858943aa6b7448ffcb02a9b5c0bf7294ca6af11f504ddde985479a9147fbb9a4d890e308aa49e80265d8d9fe29d15f1f4072c41c333e84db47dbe7ec3782ea416f3bcfb7797bf3ae3f13d3ee761a9e20fb87d95373928d300881ebb21a0cfeb67eeccdf4b5561cd02881313ade151664d06e513a989acf16e90'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 155678737 bytes downloaded
Downloaded and uncompressed: faster-rcnn-resnet50-fpn
[==================================================] 735747015 bytes downloaded
Downloaded and uncompressed: anomaly-detection
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Streaming output truncated to the last 5000 lines.
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/148.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/113.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/028.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/164.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/042.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/097.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/103.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/141.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/149.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test/Test020/049.tif
/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Te

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from PIL import Image

# Load a subset of images from a directory
def load_images_from_directory(directory, target_size=(128, 128), grayscale=False, load_fraction=0.3):
    images = []
    labels = []
    total_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif', 'tif', 'tiff')):
                total_files.append(os.path.join(root, file))

    num_files_to_load = int(len(total_files) * load_fraction)
    files_to_load = np.random.choice(total_files, num_files_to_load, replace=False)

    for img_path in files_to_load:
        try:
            if grayscale:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is None:
                    print(f"Error loading image {img_path}: Image is None")
                    continue
                img = cv2.resize(img, target_size)
                img = np.expand_dims(img, axis=-1)
            else:
                img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
                img = tf.keras.preprocessing.image.img_to_array(img)
            img = img / 255.0
            images.append(img)
            label = 0 if 'Train' in root else 1  # Simple logic to assign labels
            labels.append(label)
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
    return np.array(images), np.array(labels)

# Define directories
normal_dir = "/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train"
anomaly_dir = "/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Test"

# Load images (30% of the total data)
normal_images, normal_labels = load_images_from_directory(normal_dir, target_size=(128, 128), load_fraction=0.03)
anomaly_images, anomaly_labels = load_images_from_directory(anomaly_dir, target_size=(128, 128), load_fraction=0.03)

print(f"Number of normal images loaded: {len(normal_images)}")
print(f"Normal Images shape: {normal_images.shape}")
print(f"Normal Labels shape: {normal_labels.shape}")
print(f"Number of anomaly images loaded: {len(anomaly_images)}")
print(f"Anomaly Images shape: {anomaly_images.shape}")
print(f"Anomaly Labels shape: {anomaly_labels.shape}")

# Combine normal and anomaly images
all_images = np.concatenate([normal_images, anomaly_images], axis=0)
all_labels = np.concatenate([normal_labels, anomaly_labels], axis=0)

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(all_images, all_labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Validation set shape: {X_val.shape}, {y_val.shape}")
print(f"Test set shape: {X_test.shape}, {y_test.shape}")


Number of normal images loaded: 204
Normal Images shape: (204, 128, 128, 3)
Normal Labels shape: (204,)
Number of anomaly images loaded: 276
Anomaly Images shape: (276, 128, 128, 3)
Anomaly Labels shape: (276,)
Training set shape: (288, 128, 128, 3), (288,)
Validation set shape: (96, 128, 128, 3), (96,)
Test set shape: (96, 128, 128, 3), (96,)


In [ ]:
import torch
import urllib.request

# URL for the ResNet50 backbone weights
url = "https://download.pytorch.org/models/resnet50-0676ba61.pth"

# Destination path to save the weights
destination = "resnet50-0676ba61.pth"

# Download the weights
urllib.request.urlretrieve(url, destination)

# Verify the download
if torch.load(destination):
    print("ResNet50 backbone weights downloaded successfully.")
else:
    print("Failed to download the weights.")


ResNet50 backbone weights downloaded successfully.


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
import os

class CustomDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        import glob
        self.imgs = []
        for ext in ('*.png', '*.jpg', '*.jpeg', '*.bmp', '*.gif', '*.tif', '*.tiff'):
            self.imgs.extend(glob.glob(os.path.join(root, '**', ext), recursive=True))

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        try:
            img = Image.open(img_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            return None, None # Return None if image loading fails

        width, height = img.size  # Get dimensions before converting to tensor

        if self.transforms:
            img = self.transforms(img)

        target = {
            "boxes": torch.tensor([[0, 0, width, height]], dtype=torch.float32),
            "labels": torch.tensor([1], dtype=torch.int64),
            "masks": torch.zeros((1, height, width), dtype=torch.uint8),
            "image_id": torch.tensor([idx]),
            "area": torch.tensor([width * height], dtype=torch.float32),
            "iscrowd": torch.tensor([0], dtype=torch.int64)
        }

        return img, target

    def __len__(self):
        return len(self.imgs)

# Define your transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((800, 800))
])

# Adjusted DataLoader parameters
dataset = CustomDataset(root='/kaggle/input/anomaly-detection/UCSD_Anomaly_Dataset.v1p2/UCSDped1/Train/Train007', transforms=transform)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=2, pin_memory=True, collate_fn=lambda x: tuple(zip(*x)))

# Model preparation
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
num_classes = 2
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
model.rpn.anchor_generator.sizes = ((32,), (64,), (128,), (256,), (512,))
model.rpn.anchor_generator.aspect_ratios = ((0.5, 1.0, 2.0),) * len(model.rpn.anchor_generator.sizes)
model.to(device)

# Use the AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

# Training loop without mixed precision if CUDA is not available
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in data_loader:
        # Handle potential None returns from __getitem__
        if images is None or targets is None:
            continue
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {losses.item():.4f}")

# Save the trained model
torch.save(model.state_dict(), "fasterrcnn_model.pth")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/10], Loss: 0.0875


In [ ]:
# Load pre-trained (or fine-tuned) Faster R-CNN model
model.load_state_dict(torch.load("fasterrcnn_finetuned.pth"))
model.eval()

# CNN Model Definition
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)  # Assuming input is 128x128 after pooling
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        if x.size(2) > 1 and x.size(3) > 1:  # Check if pooling is possible
            x = self.pool(x)
        x = F.relu(self.conv2(x))
        if x.size(2) > 1 and x.size(3) > 1:  # Check if pooling is possible
            x = self.pool(x)

        x = x.view(x.size(0), -1)
        self.fc1 = nn.Linear(x.size(1), 128)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# MLP Model Definition
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)  # Binary classification

    def forward(self, x):
        if not hasattr(self, 'fc1'):
            self.fc1 = nn.Linear(x.size(1), 128).to(x.device)  # Initialize fc1 dynamically based on input size
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Dummy models for demonstration purposes (not trained)
def classify_anomaly_with_cnn(cnn_model, image_tensor):
    with torch.no_grad():
        return cnn_model(image_tensor)

def classify_anomaly_with_mlp(mlp_model, image_tensor):
    with torch.no_grad():
        batch_size = image_tensor.size(0)
        image_tensor = image_tensor.view(batch_size, -1)  # Flatten the input
        image_tensor = F.interpolate(image_tensor.unsqueeze(1), size=432, mode='linear', align_corners=False).squeeze(1)  # Resize the flattened tensor to 432
        return mlp_model(image_tensor)

# Visualize results
def visualize_detection(image, boxes, labels, title):
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    ax = plt.gca()
    for i, box in enumerate(boxes):
        xmin, ymin, xmax, ymax = box
        rect = plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color='red', linewidth=2)
        ax.add_patch(rect)
        label_name = coco_labels.get(labels[i].item(), 'unknown')  # Default to 'unknown' if label not found
        ax.text(xmin, ymin, label_name, fontsize=15, bbox=dict(facecolor='yellow', alpha=0.5))
    plt.title(title)
    plt.show()

# Use fine-tuned Faster R-CNN for object detection
def detect_anomalies_with_fasterrcnn(model, image):
    model.eval()
    with torch.no_grad():
        predictions = model([image])
        boxes = predictions[0]['boxes']
        scores = predictions[0]['scores']
        labels = predictions[0]['labels']
    return boxes, labels, scores
